Retrieve exchange rate from various providers:
- CoinGekco
- Binance
- Kraken
- ChainLink

In [1]:
%%capture --no-display
%pip install python-dotenv
%pip install web3
%pip install pandas
%pip install plotly

In [2]:
%load_ext dotenv
%dotenv

In [3]:
import requests
import pandas as pd
import plotly.graph_objects as go

BTC/USD exchange rate from CoinGecko

In [4]:
url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
response = requests.get(url)
data = response.json()
data

{'bitcoin': {'usd': 69180}}

BTC/USDC exchange rate from Binance

In [5]:
url = "https://api.binance.com/api/v3/ticker/price"
parameters = {
  'symbol':'BTCUSDC'  
}

response = requests.get(url, params=parameters)
data = response.json()
data

{'symbol': 'BTCUSDC', 'price': '69200.00000000'}

BTCUSDT exchange rate from kraken

In [6]:
parameters = {
  'pair': 'BTCUSDT',
  'interval': '240',
  'since': '1704038400'
}
response = requests.get('https://api.kraken.com/0/public/OHLC?', params=parameters)
data = response.json()
data

{'error': [],
 'result': {'XBTUSDT': [[1706328000,
    '41822.8',
    '41877.8',
    '41624.9',
    '41711.7',
    '41749.0',
    '14.46644084',
    253],
   [1706342400,
    '41706.5',
    '41783.9',
    '41414.4',
    '41763.0',
    '41587.9',
    '10.84189459',
    379],
   [1706356800,
    '41765.7',
    '41896.8',
    '41721.8',
    '41813.1',
    '41844.4',
    '73.59693039',
    468],
   [1706371200,
    '41821.0',
    '41947.5',
    '41756.0',
    '41917.6',
    '41875.8',
    '7.47360955',
    256],
   [1706385600,
    '41926.9',
    '42182.1',
    '41926.7',
    '42120.0',
    '42066.0',
    '21.68001787',
    381],
   [1706400000,
    '42130.9',
    '42225.0',
    '41969.7',
    '42213.6',
    '42157.4',
    '16.37042236',
    236],
   [1706414400,
    '42213.6',
    '42800.0',
    '42213.6',
    '42391.6',
    '42523.5',
    '23.85051884',
    815],
   [1706428800,
    '42409.2',
    '42678.9',
    '42336.1',
    '42475.5',
    '42555.6',
    '36.03335017',
    521],
   [17

In [7]:
ohlc = data['result']['XBTUSDT']

In [8]:
df = pd.DataFrame (ohlc, columns = ['timestamp', 'open', 'high', 'low', 'close', 'current','volume', 'trades'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit = 's')
df.head()

,timestamp,open,high,low,close,current,volume,trades
0,2024-01-27 04:00:00,41822.8,41877.8,41624.9,41711.7,41749.0,14.46644084,253
1,2024-01-27 08:00:00,41706.5,41783.9,41414.4,41763.0,41587.9,10.84189459,379
2,2024-01-27 12:00:00,41765.7,41896.8,41721.8,41813.1,41844.4,73.59693039,468
3,2024-01-27 16:00:00,41821.0,41947.5,41756.0,41917.6,41875.8,7.47360955,256
4,2024-01-27 20:00:00,41926.9,42182.1,41926.7,42120.0,42066.0,21.68001787,381


In [9]:
fig = go.Figure()

fig.add_trace(go.Candlestick(
  x = df.timestamp,
  open = df[('open')],
  high = df[('high')],
  low = df[('low')],
  close = df[('close')],
  showlegend = False)
)
fig.update_layout(xaxis_rangeslider_visible = False)
fig.show()

BTC/USD from Chainlink

In [10]:
import os

infuraApiKey = os.environ.get("INFURA_API_KEY")

In [11]:

from web3 import Web3

infura_url = f"https://mainnet.infura.io/v3/{infuraApiKey}"
web3 = Web3(Web3.HTTPProvider(infura_url))
web3.is_connected()

True

In [12]:
import json

sc_address="0xF4030086522a5bEEa4988F8cA5B36dbC97BeE88c"

with open ("./abi/chainlink_btcusd.json") as file:
    abi = json.load(file)

contract = web3.eth.contract(address=sc_address, abi=abi)

contract.all_functions()

[<Function decimals()>,
 <Function description()>,
 <Function getRoundData(uint80)>,
 <Function latestRoundData()>,
 <Function version()>]

In [13]:
decimals = contract.functions.decimals().call()
latestRoundData = contract.functions.latestRoundData().call()
decimals, latestRoundData

(8,
 [110680464442257325954,
  6915314537765,
  1716685835,
  1716685835,
  110680464442257325954])

In [14]:
price = latestRoundData[1] * 10 ** -decimals
price

69153.14537765